# Hot-Pow
### Investigating SnowEx Temperature Observations at Grand Mesa, Colorado, Winter 2019-2020

#### Learning objectives
* **Data Wrangling**
    * How can we pull in new external data sources and format these for use
* **Data Visualization**
    * Timeseries plotting with matplotlib and other interactive plotters (e.g., Holoviz)
* **Advanced Geospatial Visualization**
    * GIS style and multi-dimensional plotting
* **Exploring Science Questions using Thermal Data**
    * *Add more here which gets echoed under summary* 
    * How does SnowEx data (especially temperatures) compare to the ERA5 reanalysis product? 
    * How do near surface temperatures taken at snow puts compare to other thermal datasets? and more...

In [1]:
# add flowchart for going from in-situ to airborne to model data

## Grand Mesa, Colorado, Winter 2019-2020

In [ ]:
# add image of grand mesa

## Snow pit temperature data
*Add brief description....*    

In [3]:
# add map of snowpit distribution-- from Dillon?

#### Data wrangling

We considered:

* Grand Mesa IOP snow pit temperature data
* Grand Mesa Time Series snow pit temperature data
* COVID quarantine extended snow pit temperature data (which will be available with SnowEx data following QA/QC)

Our hackweek exploration (as Python/Jupyter/Git newbies): can we collect, transform, and integrate these data such that we can look at a timeline of snow temperatures on the Mesa from a winter snowpack, through warming and ripening?


In [ ]:
# show some screenshots of our data wrangling here

#### Visualization

In [ ]:
 # this could be a great opportunity to look at some of Aji's work

#### Takeaways

* Integrating data from different sources poses different challenges
    * Reorienting and matching data frames took some troubleshooting
    * Pandas, Geopandas, and Holoviz
* A season-long integration of snow pit data can be useful to compare with other observations of snow temperatures

## Thermal IR
*Add brief description....*  

#### Data wrangling
*Add description of where the data came from and what needed to be done to get it analysis ready.*   
*Feel free to just load the preprocessed file from disk, instead of going through all the preprocessing steps.*

#### Visualization

#### Takeaways

## ERA5 vs. SNOTEL vs. SnowEx Temperature comparison

### How does modeled ERA5 climate data compare to SNOTEL and SnowEx measurements?   

#### Variables of interest:

- 2m_temperature
- snow_albedo
- snow_density
- snow_depth
- snow_depth_water_equivalent
- temperature_of_snow_layer
- total_precipitation   


In this example we will pull in the [ERA5-Land hourly data from 1981 to present](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=form)  

<p align="center">
<img src="https://datastore.copernicus-climate.eu/c3s/published-forms/c3sprod/reanalysis-era5-land/overview.png" width=300>   
</p>

| DATA DESCRIPTION  |  |
| ------------- | ------------- |
| **Data type**  | Gridded |
| **Projection**  | Regular latitude-longitude grid |
| **Horizontal coverage**  | Global |
| **Horizontal resolution**  | 0.1° x 0.1°; Native resolution is 9 km. |
| **Vertical coverage**  | From 2 m above the surface level, to a soil depth of 289 cm. |
| **Vertical resolution**  | 4 levels of the ECMWF surface model: Layer 1: 0 -7cm, Layer 2: 7 -28cm, Layer 3: 28-100cm, Layer 4: 100-289cm Some parameters are defined at 2 m over the surface. |
| **Temporal coverage**  | January 1981 to present |
| **Temporal resolution**  | Hourly |
| **File format**  | GRIB / NetCDF |
| **Update frequency**  | Monthly with a delay of about three months relatively to actual date. |    

 
Additional products and documentation can be found here:   
[ERA5-Land: documentation](https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation)      
[ERA5-Land monthly averaged data from 1981 to present](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land-monthly-means?tab=form)     
[ERA5 monthly averaged data on pressure levels from 1950 to 1978 (preliminary version)](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels-monthly-means-preliminary-back-extension?tab=overview)    
[ERA5 hourly data on pressure levels from 1950 to 1978 (preliminary version)](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels-preliminary-back-extension?tab=overview)    


#### Data wrangling
*Add description of where the data came from and what needed to be done to get it analysis ready.*   
*Feel free to just load the preprocessed file from disk, instead of going through all the preprocessing steps.*

#### Visualization

#### Takeaways

## Conopy shadows and impact on snow surface temperature
*Add brief description....*  

#### Data wrangling
*Add description of where the data came from and what needed to be done to get it analysis ready.*   
*Feel free to just load the preprocessed file from disk, instead of going through all the preprocessing steps.*

#### Visualization

#### Takeaways

## Summary
*Summary and takeaways*